In [ ]:
%pip install pandas scikit-learn transformers torch seaborn

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import RobertaTokenizerFast, RobertaModel
from tqdm.auto import tqdm

# 1.CONFIGURATION
BASE = "C:/Users/indur/OneDrive - University of Westminster/GitHub/FYP_Project/Models"
DATA_PATH    = f"{BASE}/Ai_Genuine_Reviews/FinalDataSet/filtered_reviews.csv"
MODEL_OUT    = f"{BASE}/ModelTesting/Ai_Genuine_ReviewsTest/Roberta_Lstm/roberta_lstm.pt"
NUM_EPOCHS   = 5
PATIENCE     = 3
BATCH_SIZE   = 16
MAX_LENGTH   = 256
LR           = 2e-5
WEIGHT_DECAY = 0.01

# 2.LOAD & SPLIT
df = pd.read_csv(DATA_PATH)

print("Dataset size:", len(df))
print("Label distribution:\n", df['label'].value_counts())
print("Duplicate reviews:", df['clean_review'].duplicated().sum())

df = df.drop_duplicates(subset=['clean_review'], keep='first')
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df.label, random_state=42)
print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))
if len(val_df) < 100:
    print("Warning: Validation set is small (<100 samples), which may lead to unreliable metrics.")

train_reviews = set(train_df['clean_review'])
val_reviews = set(val_df['clean_review'])
print("Overlapping reviews:", len(train_reviews.intersection(val_reviews)))

# 3.DEVICE SETUP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if torch.cuda.is_available():
    print("GPU Memory Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
    torch.cuda.empty_cache()

# 4.TOKENIZER & DATASET
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

class HybridDataset(Dataset):
    def __init__(self, texts, labels):
        enc = tokenizer(texts.tolist(),
                        truncation=True,
                        padding="max_length",
                        max_length=MAX_LENGTH,
                        return_tensors="pt")
        self.input_ids      = enc.input_ids.to(device)
        self.attention_mask = enc.attention_mask.to(device)
        self.labels         = torch.tensor(labels.values, dtype=torch.long).to(device)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids":      self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels":         self.labels[idx],
        }

train_ds = HybridDataset(train_df.clean_review, train_df.label)
val_ds   = HybridDataset(val_df.clean_review,   val_df.label)
train_ld = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_ld   = DataLoader(val_ds,   batch_size=BATCH_SIZE*2)

# 5.MODEL DEFINITION
class RobertaLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta    = RobertaModel.from_pretrained("roberta-base").to(device)
        hid_size        = self.roberta.config.hidden_size
        self.lstm       = nn.LSTM(hid_size, 256, batch_first=True, bidirectional=True).to(device)
        self.dropout    = nn.Dropout(0.5)
        self.classifier = nn.Linear(2*256, 2).to(device)

    def forward(self, input_ids, attention_mask):
        seq_output, _ = self.roberta(input_ids=input_ids,
                                     attention_mask=attention_mask,
                                     return_dict=False)
        lstm_out, _   = self.lstm(seq_output)
        pooled        = lstm_out.mean(dim=1)
        return self.classifier(self.dropout(pooled))

model = RobertaLSTM().to(device)
optim = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
crit  = nn.CrossEntropyLoss().to(device)

# 6.EARLY STOPPING UTILITY
class EarlyStopping:
    def __init__(self, patience=PATIENCE, verbose=True):
        self.patience   = patience
        self.verbose    = verbose
        self.best_f1    = 0
        self.counter    = 0
        self.best_state = None

    def step(self, current_f1, model):
        if current_f1 > self.best_f1:
            self.best_f1    = current_f1
            self.counter    = 0
            self.best_state = model.state_dict()
            if self.verbose:
                print(f"New best F1: {current_f1:.4f}")
        else:
            self.counter += 1
            if self.verbose:
                print(f"No improvement. Counter: {self.counter}/{self.patience}")
        return self.counter >= self.patience

early_stopper = EarlyStopping(patience=PATIENCE, verbose=True)

# 7. TRAINING LOOP WITH PROGRESS BARS & EARLY STOPPING
for epoch in range(1, NUM_EPOCHS + 1):
    print(f"Starting Epoch {epoch}")
    # Training
    model.train()
    train_loss = 0
    train_iter = tqdm(train_ld, desc=f"Epoch {epoch} [Training]", leave=False)
    for batch in train_iter:
        optim.zero_grad()
        logits = model(batch["input_ids"], batch["attention_mask"])
        loss   = crit(logits, batch["labels"])
        loss.backward()
        optim.step()
        train_loss += loss.item()
        train_iter.set_postfix(loss=loss.item())
    avg_train_loss = train_loss / len(train_ld)

    # Validation
    model.eval()
    preds, trues = [], []
    val_iter = tqdm(val_ld, desc=f"Epoch {epoch} [Validation]", leave=False)
    for batch in val_iter:
        with torch.no_grad():
            logits = model(batch["input_ids"], batch["attention_mask"])
        preds.extend(torch.argmax(logits, dim=1).cpu().tolist())
        trues.extend(batch["labels"].cpu().tolist())
    val_f1 = f1_score(trues, preds)
    val_acc = accuracy_score(trues, preds)

    # Confusion Matrix
    cm = confusion_matrix(trues, preds)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
    plt.title(f"Confusion Matrix - Epoch {epoch}")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()

    print(f"Epoch {epoch}/{NUM_EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")

    # Early stopping check
    if early_stopper.step(val_f1, model):
        print(f"Stopping early at epoch {epoch}")
        model.load_state_dict(early_stopper.best_state)
        break

    if torch.cuda.is_available():
        print("GPU Memory Used:", torch.cuda.memory_allocated() / 1024**2, "MB")

# 8. SAVE BEST MODEL
torch.save(model.state_dict(), MODEL_OUT)
print(f"Saved best RoBERTa+LSTM model to {MODEL_OUT} (Val F1 = {early_stopper.best_f1:.4f})")